# 1) Count-based trigram model with linear interpolation

A count-based trigram model with linear-interpolation. That is an interpolation of three categorical distributions with $V$, $V^2$, and $V^3$ parameters: $$p(y_t | y_{1:t-1}) =  \alpha_1 p(y_t | y_{t-2}, y_{t-1}) + \alpha_2 p(y_t | y_{t-1}) + (1 - \alpha_1 - \alpha_2) p(y_t) $$
Try out different values of $\alpha$. 

In [0]:
!pip install -q torchtext 

import torch
import torchtext
from torchtext.vocab import Vectors
from torchtext.data.iterator import BPTTIterator
from torchtext.data import Batch, Dataset
import math


import numpy as np

from tqdm import tqdm_notebook
from collections import Counter, defaultdict

In [2]:
# Our input $x$
TEXT = torchtext.data.Field()

!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/input.txt
!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/train.5k.txt
!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/train.txt
!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/valid.txt
!curl -qO https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb.test.txt

# Data distributed with the assignment
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(
    path=".", 
    train="train.txt", validation="valid.txt", test="ptb.test.txt", text_field=TEXT)

train_iter, val_iter, test_iter = BPTTIterator.splits(
    (train, val, test), batch_size=10, device=torch.device("cuda"), bptt_len=32, repeat=False)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  185k  100  185k    0     0   605k      0 --:--:-- --:--:-- --:--:--  603k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  595k  100  595k    0     0  1431k      0 --:--:-- --:--:-- --:--:-- 1431k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4982k  100 4982k    0     0  13.0M      0 --:--:-- --:--:-- --:--:-- 13.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  390k  100  390k    0     0  1394k      0 --:--:-- --:--:-- --:--:-- 1394k
  % Total    % Received % Xferd  Average Speed   Tim

In [0]:
class TrigramLM:
  def __init__(self, unk_tok):
    print('Initializing trigram model...')
    self.unigram_counts = defaultdict(lambda: 0)
    self.bigram_counts = defaultdict(lambda: defaultdict(lambda: 0))
    self.trigram_counts = defaultdict(lambda: defaultdict(lambda: 0))
  
  def train(self, train_iter):
    for batch in tqdm_notebook(train_iter, position=0, leave=True):
      _, batch_size = batch.text.shape
      for i in range(batch_size):
        sentence = batch.text[:,i]
        for j, tok_tensor in enumerate(sentence.data):
          tok_idx = tok_tensor.item()
          self.unigram_counts[tok_idx] += 1
          self.bigram_counts[sentence.data[j-1].item()][tok_idx] += 1
          self.trigram_counts[(sentence.data[j-2].item(), sentence.data[j-1].item())][tok_idx] += 1

In [4]:
TEXT.build_vocab(train) #for whole vocab
# TEXT.build_vocab(train, max_size=1000) # for debugging
unk_tok = TEXT.vocab.stoi["<unk>"]

model = TrigramLM(unk_tok)

Initializing trigram model...


In [5]:
print('Learning counts...')
model.train(train_iter)

Learning counts...


In [0]:
def generate_probs(model, unk_tok):
  print('Recalculating qML for unigrams, bigrams, trigrams')

  print('\tCalculating unigram probabilities...')
  model.unigram_probs = dict()
  # if unk_tok not in model.unigram_counts: #unk tok has to have been seen at least once
  #   model.unigram_counts[unk_tok] = 1
  for k, v in model.unigram_counts.items():
    model.unigram_probs[k] = float(v / sum(model.unigram_counts.values()))

  print('\tCalculating bigram probabilities...')
  model.bigram_probs = dict()
  for unigram, bigram_counter in tqdm_notebook(model.bigram_counts.items(), position=0, leave=True):
    model.bigram_probs[unigram] = dict()
    # if unk_tok not in bigram_counter: #every bigram has to have seen the unknown token at least once
    #   bigram_counter[unk_tok] = 1
    for tok, count in bigram_counter.items():
      model.bigram_probs[unigram][tok] = float(count / (model.unigram_counts[unigram]))
        
  print('\tCalculating trigram probabilities...')
  model.trigram_probs = dict()
  for bigram, trigram_counter in tqdm_notebook(model.trigram_counts.items(), position=0, leave=True):
    model.trigram_probs[bigram] = dict()
    # if unk_tok not in trigram_counter: #every trigram has to have seen the unknown token at least once
    #   trigram_counter[unk_tok] = 1
    for tok, count in trigram_counter.items():
      model.trigram_probs[bigram][tok] = float(count / (model.bigram_counts[bigram[0]][bigram[1]]))
  
  return model

In [7]:
model = generate_probs(model, unk_tok)

Recalculating qML for unigrams, bigrams, trigrams
	Calculating unigram probabilities...
	Calculating bigram probabilities...



	Calculating trigram probabilities...


In [8]:
# Check our counts and probs

print('c(): {}'.format(sum(model.unigram_counts.values())))
print('c(0): {}'.format(model.unigram_counts[0]))
print('c(0|0): {}'.format(model.bigram_counts[0][0]))
print('c(0|0,0): {}'.format(model.trigram_counts[(0,0)][0]))

assert model.unigram_probs[0] == float(model.unigram_counts[0] / np.sum([x for x in model.unigram_counts.values()]))
assert model.bigram_probs[0][0] == float(model.bigram_counts[0][0] / model.unigram_counts[0])
assert model.trigram_probs[(0,0)][0] == float(model.trigram_counts[(0,0)][0] / model.bigram_counts[0][0])
assert model.trigram_probs[(2,4)][2] == float(model.trigram_counts[(2,4)][2] / model.bigram_counts[2][4])
assert sum([model.trigram_counts[(2,4)][tok] for tok in model.trigram_counts[(2,4)]]) == model.bigram_counts[2][4]
assert model.unigram_counts[0] == sum(model.bigram_counts[0].values())

c(): 929580
c(0): 45020
c(0|0): 3981
c(0|0,0): 454


In [0]:
def get_pp(X_iter, model, a_1, a_2, oov, unk_tok):
  print('Getting perplexity...')
  # Build p_x as arr of probs across the entire iter
  p_x = list()
  for batch in tqdm_notebook(X_iter, position=0, leave=True):
    _, batch_size = batch.text.shape 
    for i in range(batch_size):
      sentence = batch.text[:,i]
      for j, tok_tensor in enumerate(sentence.data):
        p_x_j = 0.
        try:
          # Assemble prob of this trigram.
          # Note unk handling defaults to 0.

          # Start with unigram
          tok_idx = tok_tensor.item()
          if tok_idx not in model.unigram_probs:
            unigram_prob = 0.
          else:
            unigram_prob = model.unigram_probs[tok_idx]
          p_x_j += (1 - a_1 - a_2) * unigram_prob

          # Then if there's an eligible bigram, add it
          if j > 0:
            tok_idx = tok_tensor.item()
            unigram_tok = sentence.data[j-1].item()
            # Attempt unk_tok replacements
            if unigram_tok not in model.bigram_probs:
              #model hasn't seen j-1 before; default to (<unk>, j)
              unigram_tok = unk_tok
            if tok_idx not in model.bigram_probs[unigram_tok]:
              #j hasn't come after j-1 before; default to (j-1/<unk>, <unk>)
              tok_idx = unk_tok
            # Then attempt constructing the bigram probability
            if tok_idx not in model.bigram_probs[unigram_tok]:
              bigram_prob = 0.
            else:
              bigram_prob = model.bigram_probs[unigram_tok][tok_idx]
            p_x_j += (a_2) * bigram_prob

          # Then if there's an eligible trigram, add it
          if j > 1:
            tok_idx = tok_tensor.item()
            bigram_tup = (sentence.data[j-2].item(), sentence.data[j-1].item())
            # Attempt unk_tok replacements
            while bigram_tup not in model.trigram_probs and bigram_tup != (unk_tok, unk_tok):
              # model hasn't seen (j-2, j-1) before
              # replace one and then the other
              for i in range(2):
                new_bigram_tup = list(bigram_tup)
                new_bigram_tup[i] = unk_tok
                bigram_tup = tuple(new_bigram_tup)
            if tok_idx not in model.trigram_probs[bigram_tup]:
              # j hasn't come after (j-2, j-1) before
              tok_idx = unk_tok
            # Then attempt constructing the trigram probability
            if tok_idx not in model.trigram_probs[bigram_tup]:
              trigram_prob = 0.
            else:
              trigram_prob = model.trigram_probs[bigram_tup][tok_idx]
            p_x_j += a_1 * trigram_prob

        except:
          print('tok_idx: ', tok_idx)
          if unigram_tok:
            print('unigram_tok', unigram_tok)
          if bigram_tup:
            print('bigram_tup: ', bigram_tup)
          raise
        p_x.append(p_x_j)
  # Convert p_x to perplexity
  pp = np.exp(np.mean(-np.log(p_x)))
  return pp, p_x

In [0]:
a_1 = 0.7
a_2 = 0.1
oov = 1e-10
unk_tok = TEXT.vocab.stoi["<unk>"]

In [0]:
train_perplexity, train_p_x = get_pp(train_iter, model, a_1, a_2, oov, unk_tok)
print('\ntrain perplexity: ', train_perplexity)

Getting perplexity...




train perplexity:  inf


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log


In [0]:
val_perplexity, val_p_x = get_pp(val_iter, model, a_1, a_2, oov, unk_tok)
print('\nval perplexity: ', val_perplexity)

Getting perplexity...




val perplexity:  inf


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log


In [0]:
test_perplexity, test_p_x = get_pp(test_iter, model, a_1, a_2, oov, unk_tok)
print('\ntest perplexity: ', test_perplexity)

Getting perplexity...




test perplexity:  inf


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log


In [0]:
def get_H(batch, model, a_1, a_2, unk_tok):
  print('Getting perplexity...')
  # Build p_x as arr of probs across the entire iter
  p_x = list()
  _, batch_size = batch.text.shape 
  for i in range(batch_size):
    sentence = batch.text[:,i]
    for j, tok_tensor in enumerate(sentence.data):
      p_x_j = 0.
      try:
        # Assemble prob of this trigram.
        # Note unk handling defaults to 0.

        # Start with unigram
        tok_idx = tok_tensor.item()
        if tok_idx not in model.unigram_probs:
          unigram_prob = 0.
        else:
          unigram_prob = model.unigram_probs[tok_idx]
        p_x_j += (1 - a_1 - a_2) * unigram_prob

        # Then if there's an eligible bigram, add it
        if j > 0:
          tok_idx = tok_tensor.item()
          unigram_tok = sentence.data[j-1].item()
          # Attempt unk_tok replacements
          if unigram_tok not in model.bigram_probs:
            #model hasn't seen j-1 before; default to (<unk>, j)
            unigram_tok = unk_tok
          if tok_idx not in model.bigram_probs[unigram_tok]:
            #j hasn't come after j-1 before; default to (j-1/<unk>, <unk>)
            tok_idx = unk_tok
          # Then attempt constructing the bigram probability
          if tok_idx not in model.bigram_probs[unigram_tok]:
            bigram_prob = 0.
          else:
            bigram_prob = model.bigram_probs[unigram_tok][tok_idx]
          p_x_j += (a_2) * bigram_prob

        # Then if there's an eligible trigram, add it
        if j > 1:
          tok_idx = tok_tensor.item()
          bigram_tup = (sentence.data[j-2].item(), sentence.data[j-1].item())
          # Attempt unk_tok replacements
          while bigram_tup not in model.trigram_probs and bigram_tup != (unk_tok, unk_tok):
            # model hasn't seen (j-2, j-1) before
            # replace one and then the other
            for i in range(2):
              new_bigram_tup = list(bigram_tup)
              new_bigram_tup[i] = unk_tok
              bigram_tup = tuple(new_bigram_tup)
          if tok_idx not in model.trigram_probs[bigram_tup]:
            # j hasn't come after (j-2, j-1) before
            tok_idx = unk_tok
          # Then attempt constructing the trigram probability
          if tok_idx not in model.trigram_probs[bigram_tup]:
            trigram_prob = 0.
          else:
            trigram_prob = model.trigram_probs[bigram_tup][tok_idx]
          p_x_j += a_1 * trigram_prob

      except:
        print('tok_idx: ', tok_idx)
        if unigram_tok:
          print('unigram_tok', unigram_tok)
        if bigram_tup:
          print('bigram_tup: ', bigram_tup)
        raise
      p_x.append(p_x_j)
  # Convert p_x to entropy
  H = np.mean(-np.log(p_x))
  return H, p_x

In [0]:
examples = list()
for i, batch in enumerate(train_iter):
  if i in [1, 7]:
    examples.append(batch)
for i, batch in enumerate(test_iter):
  if i in [117, 231]:
    examples.append(batch)

In [35]:
entropies = list()
for context in examples:
  H, _ = get_H(context, model, a_1, a_2, unk_tok)
  entropies.append(H)
entropies

Getting perplexity...
Getting perplexity...
Getting perplexity...
Getting perplexity...


[2.503588542981169, 2.5349433733878053, 3.9920512375003185, 3.857189940545878]